<a href="https://colab.research.google.com/github/mehedihasan-cse/ML-Projects/blob/main/Hate_Speech_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [475]:
import numpy as np  #To work with multi-dimentional array and matrix
import pandas as pd  #Data Manipulation, Data strcutre
import matplotlib.pyplot as plt #Data Visualization
import seaborn as sns
import nltk
import string
%matplotlib inline

In [476]:
df = pd.read_csv("/content/bengali_hate_v2.0.csv")

In [509]:
df.tail(7)

,text,label,target
5691,তুই তো নাস্তিক সেটা আমরা সবাই জানি তুই তো আরো ...,Religious,2
5692,সত্যি করে বল ডাইনোসর বিলুপ্ত হয়ে গেছিল নাকি তো...,Geopolitical,3
5693,জুতা যতই দামি হোক তার স্থান পায়ে তার স্থান কখ...,Personal,0
5694,আমাদের দেশ ছোট বলে ভারতের চেয়ে পিছিয়ে না হয় ...,Geopolitical,3
5695,হিন্দুদের মধ্যে এগুলো হয় এটাই তার বাস্তব প্রমা...,Personal,0
5696,দেখলে মনে হয় শালী একটা অস্ট্রেলিয়ান গাভী,Personal,0
5697,ভাই আমি আপনার সাথে একমত। আর নারী নেতৃত্বে থাকত...,Political,1


In [478]:
df.columns.values

array(['text', 'label', 'target'], dtype=object)

In [479]:
df.value_counts()

,,,count
text,label,target,
অন্ন কেহু না সব রেন্ডিয়ার দাদা বাবুরা। চিন্তার কারন নাই।,Geopolitical,3,1
ভাই আপনি তাকে কেন বলছেন না যে তুমি উর্দুতে কথা বল আমি যেভাবে ধর্ম গ্রন্থ থেকে রেফারেন্স দিয়ে কথা বলতেছি তুমি রেফারেন্সে কথা বল এটা বলুন,Religious,2,1
ভাই তোর সাফল্যের রহস্যটা কি? আমি পর্ণ ওয়েবসাইট ব্যান করি নাই,Personal,0,1
ভাই তোর মতো মা চোদা আর দেখিনি,Personal,0,1
ভাই তোমারে তো চুরের মত লাগে.,Personal,0,1
...,...,...,...
কিছু কুচুটে লোকের অসৎ উদ্দেশ্য চরিতার্থ করার জন্য আপনি ভিডিও প্রচার করে যাওয়া থেকে বিরত থাকার অনুরোধ করছিভালো থাকুন,Personal,0,1
"কি সব বাল সাল, সোনিকা ধুর ক্ষ্যাত",Personal,0,1
"কি যে ওর কন্ঠ সুর শুনলে পাড়েই বমি আসে, হিমেশের সালমান খান বেশি ফেমাস হওয়ার জন্য এইসব পাগলদের কে প্রশ্রয় দেয়",Personal,0,1


# **Data Pre-Processing**

In [480]:
df.drop_duplicates(inplace =True)
df = df.replace('\n','',regex= True)
df = df.replace(',,',',',regex= True)

In [481]:
df

,text,label,target
0,বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না - পোল...,Personal,0
1,এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে ...,Political,1
2,পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস...,Geopolitical,3
3,ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভা...,Geopolitical,3
4,খানকির পুলা মালায়নদের মেরে সাফা করে ফেল,Personal,0
...,...,...,...
5693,জুতা যতই দামি হোক তার স্থান পায়ে তার স্থান কখ...,Personal,0
5694,"আমাদের দেশ ছোট বলে ভারতের চেয়ে পিছিয়ে, না হয়...",Geopolitical,3
5695,হিন্দুদের মধ্যে এগুলো হয় এটাই তার বাস্তব প্রমা...,Personal,0
5696,দেখলে মনে হয় শালী একটা অস্ট্রেলিয়ান গাভী,Personal,0


In [482]:
punct=string.punctuation

In [483]:
def remove_punct(x):
    if isinstance(x, str):  # Check if x is a string
        return x.translate(str.maketrans("", "", punct))
    else:
        return x  # Return the original value if it's not a string

df['text'] = df['text'].apply(lambda x: remove_punct(x))

In [484]:
df

,text,label,target
0,বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না পোলা...,Personal,0
1,এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে ...,Political,1
2,পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস...,Geopolitical,3
3,ভারতের অর্থনীতি নিয়ে আপনাদের ভাবতে হবে না। ভা...,Geopolitical,3
4,খানকির পুলা মালায়নদের মেরে সাফা করে ফেল,Personal,0
...,...,...,...
5693,জুতা যতই দামি হোক তার স্থান পায়ে তার স্থান কখ...,Personal,0
5694,আমাদের দেশ ছোট বলে ভারতের চেয়ে পিছিয়ে না হয় ...,Geopolitical,3
5695,হিন্দুদের মধ্যে এগুলো হয় এটাই তার বাস্তব প্রমা...,Personal,0
5696,দেখলে মনে হয় শালী একটা অস্ট্রেলিয়ান গাভী,Personal,0


In [485]:
df.isna().sum()

,0
text,0
label,0
target,0


In [486]:
df = df.fillna(method= 'pad')

In [487]:
df.isna().sum()

,0
text,0
label,0
target,0


In [488]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5698 entries, 0 to 5697
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5698 non-null   object
 1   label   5698 non-null   object
 2   target  5698 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 133.7+ KB


In [489]:
df.value_counts()

,,,count
text,label,target,
খানকির পোলা রোহিঙ্গারা বাংলাদেশে ঢুকলে কি তোর কোন সমস্যা,Personal,0,2
অন্ন কেহু না সব রেন্ডিয়ার দাদা বাবুরা। চিন্তার কারন নাই।,Geopolitical,3,1
ভাই ছোট আজাদ চুদনাপানা না করলেই কি না,Personal,0,1
ভাই তোমাকে বলছি তুমি রানু দিয়ে নিয়ে কোন ভিডিও না বানিয়ে অন্য কিছু ভিডিও বানাও কারণ এই রানুদির ভিডিও তো দূরের কথা এর কথা শুনলে আমার মাথা গরম হয়ে,Personal,0,1
ভাই তোমাকে অনেক ধন্যবাদ ওদের মা মেয়ের কথা গুলো তুলে ধরার জন্য মেয়েকেও এক জায়গাতে ভাঙ্গা প্লেট দিয়ে বসিয়ে দাও,Personal,0,1
...,...,...,...
কি সব বাল সাল সোনিকা ধুর ক্ষ্যাত,Personal,0,1
কি যে ওর কন্ঠ সুর শুনলে পাড়েই বমি আসে হিমেশের সালমান খান বেশি ফেমাস হওয়ার জন্য এইসব পাগলদের কে প্রশ্রয় দেয়,Personal,0,1
কি যে আউল ফাউল ইংলিশ বলিস তোরা।বাংলাটাই আগে ঠিক মতো বলা শিখ হারামজাদা,Personal,0,1


In [490]:

#Drop column --> axis= 1 ; row --> axis= 0
#x = df.drop(columns = ['target','label'], axis= 1)
#y = df[['target','label']]
x= df.text.values
y= df.target.values

In [491]:
print(x)

['বৌদির দুধ দেকে তো আমার ই চোখ ঠিক ছিলো না  পোলাপান এর চোখ কিভাবে ঠিক থাকবে'
 'এই সরকার কে যারা নির্লজ্জের মত সাপোর্ট দিয়েছে বছরের পর বছর তাদের আরো এমন রাস্তায় রাস্তায় কাঁদতে হবে '
 'পিলখানা হত্যাকান্ড বাংলাদেশের প্রতিরক্ষা ব্যবস্থা ধ্বংসের জন্য ভারতের প্রত্যক্ষ সহযোগিতায় এই হত্যাকা ঘটানো হয়েছিল '
 ... 'হিন্দুদের মধ্যে এগুলো হয় এটাই তার বাস্তব প্রমাণ সবাই মতামত দিন '
 'দেখলে মনে হয় শালী একটা অস্ট্রেলিয়ান গাভী '
 'ভাই আমি আপনার সাথে একমত। আর নারী নেতৃত্বে থাকতে চাই না। ভিপি নুর অথবা এর মত সৎ ও সাহসী কোন একজন নেতা আমাদের চাই।']


In [492]:
print(y)

[0 1 3 ... 0 0 1]


In [493]:
from sklearn.model_selection import train_test_split

In [494]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3)

In [495]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [496]:
cv= CountVectorizer()
x_train = cv.fit_transform(xtrain)
x_test = cv.transform(xtest)

## **Model Algorithm**

In [497]:
model = MultinomialNB()
model.fit(x_train,ytrain)

MultinomialNB()

In [498]:
labels = model.score(x_test, ytest)
print(labels)

0.6274853801169591


**TF-IDF**

In [499]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [500]:
vectorizer = TfidfVectorizer()

x_train = vectorizer.fit_transform(xtrain)
x_test = vectorizer.transform(xtest)

In [501]:
model = MultinomialNB()
model.fit(x_train,ytrain)

MultinomialNB()

In [502]:
labels = model.score(x_test, ytest)
print(labels)

0.5666666666666667


In [503]:
from sklearn.linear_model import LogisticRegression

In [504]:
model = LogisticRegression()
model.fit(x_train, ytrain)

LogisticRegression()

In [505]:
labels = model.score(x_test, ytest)
print(labels)

0.6596491228070176


# **Draft Code**

In [506]:
pip install bnlp_toolkit

In [507]:
from wordcloud import WordCloud
from bnlp import NLTKTokenizer

ImportError: cannot import name 'triu' from 'scipy.linalg' (/usr/local/lib/python3.10/dist-packages/scipy/linalg/__init__.py)

In [ ]:
category_column = 'text'



In [ ]:
text_data = ' '.join(df[category_column].dropna().astype(str))

In [ ]:
print(text_data[:500])

In [ ]:
# Initialize the tokenizer
bnltk = NLTKTokenizer()

# Tokenize the text data
words = bnltk.word_tokenize(text_data)

# Join the tokenized words back into a single string
processed_text_data = ' '.join(words)

bengali_font_path = '/content/SutonnyMJ Regular.ttf'

wordcloud = WordCloud(font_path=bengali_font_path, width=800, height=400, background_color='white').generate(text_data)


In [ ]:
print(text_data)

In [ ]:
# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()
